<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Wed May 13 16:57:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-13T16-58-21'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 64 (delta 1), reused 39 (delta 0), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-05-13T16-58-21
total 54
-rw-------  1 root root 35149 May 13 16:58 LICENSE
drwx------  2 root root  4096 May 13 16:58 notebooks
-rw-------  1 root root  7506 May 13 16:58 README.md
drwx------  2 root root  4096 May 13 16:58 requirements
drwx------ 10 root root  4096 May 13 16:58 src


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 1.4MB 12.7MB/s 
     |████████████████████████████████| 4.3MB 22.3MB/s 
     |████████████████████████████████| 460kB 39.1MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 112kB 30.6MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=96541730d152c220d0b89db47cf915007351c4ba1c42437628315f9a52d8f032
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=73a8986f26f4a079bcb88ca73115e67b5a274090ba0dd17c78887a1eaf22d6e0
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql:

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-05-13T16-58-21/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_model_dir': 'models/saved/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 128,
  'output_vocab_size': 6000,
  'output_embedding_dim': 128,
  'latent_dim': 512,
  'learning_rate': 0.0001,
  'epochs': 150,
  'batch_size': 512,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=150, eval_averaging='macro', file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', file_model_dir='models/saved/baseline/', input_embedding_dim=128, input_vocab_size=20000, latent_dim=512, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=128, output_vocab_size=6000, random_seed=1)


wandb: Wandb version 0.8.36 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "seq2_seq_attention"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  3872768   
_________________________________________________________________
decoder (Decoder)            multiple                  6993776   
_________________________________________________________________
f1_score (F1Score)           multiple                  18000     
_________________________________________________________________
sparse_categorical_accuracy  multiple                  2         
_________________________________________________________________
f1_score (F1Score)           multiple                  18000     
_________________________________________________________________
sparse_categorical_accuracy  multiple                  2         
Total 

KeyboardInterrupt: ignored

Error in callback <function _init_jupyter.<locals>.cleanup at 0x7fefac9d4a60> (for post_run_cell):


KeyboardInterrupt: ignored